<a href="https://colab.research.google.com/github/vikas123456778009/Covid19web/blob/master/Ai_trends_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
!ls

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [5]:
spark = SparkSession.builder.appName("Transactions").getOrCreate()

In [6]:
import pandas as pd
df = pd.read_fwf('txns.txt')
df.to_csv('txns.csv')

In [7]:
!ls


sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz  txns.csv  txns.txt


In [12]:
transactions_df = spark.read.csv("/content/txns.txt", header=False, inferSchema=True)

In [13]:
transactions_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)



In [14]:
columns = ["txnno", "txndate", "custid", "amount", "product", "category", "city", "state", "spendby"]

In [15]:
transactions_df = transactions_df.toDF(*columns)

In [17]:
total_transactions_per_customer = transactions_df.groupBy("custid").count()

In [18]:
total_transactions_per_customer.show()

+-------+-----+
| custid|count|
+-------+-----+
|4000009|    6|
|4000001|    8|
|4000006|    5|
|4000005|    5|
|4000008|   10|
|4000004|    5|
|4000003|    3|
|4000010|    6|
|4000007|    6|
|4000002|    6|
+-------+-----+



In [19]:
customer_total_spend = transactions_df.groupBy("custid").sum("amount")

In [21]:
customer_total_spend.show()

+-------+------------------+
| custid|       sum(amount)|
+-------+------------------+
|4000009|            457.83|
|4000001|            651.05|
|4000006|            539.38|
|4000005|            325.15|
|4000008|            859.42|
|4000004|            337.06|
|4000003| 527.5899999999999|
|4000010|447.09000000000003|
|4000007| 699.5500000000001|
|4000002|            706.97|
+-------+------------------+



In [23]:
credit_transactions = transactions_df.filter(transactions_df["spendby"] == "credit")

In [24]:
credit_customers = credit_transactions.select("custid").distinct().count()


In [25]:
print(f"Total Customers by Credit Card: {credit_customers}")

Total Customers by Credit Card: 10


In [26]:
cash_transactions = transactions_df.filter(transactions_df["spendby"] == "cash")


In [27]:
cash_customers = cash_transactions.select("custid").distinct().count()


In [28]:
print(f"Total Customers by Cash: {cash_customers}")

Total Customers by Cash: 4


In [34]:
transactions_df.show(64)

+-----+----------+-------+------+--------------------+--------------------+--------------+--------------+-------+
|txnno|   txndate| custid|amount|             product|            category|          city|         state|spendby|
+-----+----------+-------+------+--------------------+--------------------+--------------+--------------+-------+
|    0|06-26-2011|4000001| 40.33|  Exercise & Fitness|Cardio Machine Ac...|   Clarksville|     Tennessee| credit|
|    1|05-26-2011|4000002|198.44|  Exercise & Fitness|Weightlifting Gloves|    Long Beach|    California| credit|
|    2|06-01-2011|4000002|  5.58|  Exercise & Fitness|Weightlifting Mac...|       Anaheim|    California| credit|
|    3|06-05-2011|4000003|198.19|          Gymnastics|    Gymnastics Rings|     Milwaukee|     Wisconsin| credit|
|    4|12-17-2011|4000002| 98.81|         Team Sports|        Field Hockey|   Nashville  |     Tennessee| credit|
|    5|02-14-2011|4000004|193.63|  Outdoor Recreation|Camping & Backpac...|       Chicag